In [23]:
import spiceypy as sp
# https://theskylive.com/venus-info
# wrt means With Respect To


In [24]:
# Computer position of earth at current time
import datetime as dt

today_utc = dt.datetime.today()

today_utc = today_utc.strftime('%Y-%m-%dT00:00:00')
print(f'Todays date: {today_utc}')

Todays date: 2022-12-21T00:00:00


Error... We need the lsk Leapsecond kernal!

Essentially we need to download extra files to work with
certain data/missions.
These kernals will provide us the correct space-time information for certain bodies in space. 

We will also need the Earth kernal.
spk kernal

https://naif.jpl.nasa.gov/pub/naif/generic_kernels

In [35]:
# Install the kernals
sp.furnsh(r'../kernals/lsk/naif0012.tls') # Earth time
sp.furnsh(r'../kernals/spk/de432s.bsp') # Distance of Earth and Sun

In [26]:
# Convert from UTC to Ephemeris time
# Ephemeris time: used for space time data
# Because time changes in space, this 
et_today_midnight = sp.str2et(today_utc)
print(f'Todays date in Ephemeris time: {et_today_midnight}')

Todays date in Ephemeris time: 724852869.1835989


In [27]:
# Compute current position of Earth
# Must find the ID of each body
# https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/req/naif_ids.html
# 399; center of Earth
# 10; center of the Sun
# ECLIPJ2000; ref... the ecliptic plane of Earth
earth_vector_wrt_sun, earth_sun_light_time = sp.spkgeo(targ=399, et=et_today_midnight, ref='ECLIPJ2000', obs=10)
# earth_state_vector_sun returns the xyz position and xyz velocity

print(f'Earth Position: ({earth_vector_wrt_sun[0]}, {earth_vector_wrt_sun[1]}, {earth_vector_wrt_sun[2]}) km')
print(f'Earth Velocity: ({earth_vector_wrt_sun[3]}, {earth_vector_wrt_sun[4]},{earth_vector_wrt_sun[5]}) km/s')
print(f'Time it takes for light to reach Earth: {earth_sun_light_time} (units?)')

Earth Position: (3178210.32701914, 147148332.09295017, -7542.353214658797) km
Earth Velocity: (-30.27646671436553, 0.5365275625336573,0.0012423384850258623) km/s
Time it takes for light to reach Earth: 490.94847764488924 (units?)


In [28]:
# Distance from Earth to sun
from math import sqrt 
earth_sun_distance = sqrt(
earth_vector_wrt_sun[0]**2 
+ earth_vector_wrt_sun[1]**2
+ earth_vector_wrt_sun[2]**2)
print(f'Earth is {earth_sun_distance} km away from the Sun.')

Earth is 147182650.8645194 km away from the Sun.


In [29]:
# Convert to Astronomical Units (AU)
earth_sun_distance_au = sp.convrt(earth_sun_distance, 'km', 'au')
print(f'Earth is {earth_sun_distance_au} AU away from the Sun.')

Earth is 0.9838552531579384 AU away from the Sun.


In [30]:
import numpy as np

# Convert list to numpy array
earth_state_wrt_sun = np.array(earth_vector_wrt_sun)

# Distance of the Earth to the Sun using numpy linear algebra function
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])

# First, we compute the actual orbital speed of the Earth around the Sun
earth_orbital_speed_wrt_sun = np.linalg.norm(earth_state_wrt_sun[3:])
print(f'Earth is traveling at {earth_orbital_speed_wrt_sun} km/s') 

Earth is traveling at 30.28122025405923 km/s


In [46]:
# Prove the velocity of Earth
# Now let's compute the theoretical expectation. 
# First, we load a pck file that contains
# miscellanoeus information, like the G*M values for different objects

# Fetch the mass of the sun
sp.furnsh(r'../kernals/pck/gm_de431.tpc')
_, GM_SUN = sp.bodvcd(bodyid=10, item='GM', maxn=1) 

# Compute the orbital speed
vel_orbital_func = lambda Gm, r: np.sqrt(Gm/r)
earth_orbital_speed_wrt_sun_theoretical = vel_orbital_func(GM_SUN[0], earth_sun_distance)
print(f'Theoretical orbital speed of the Earth around Sun: {earth_orbital_speed_wrt_sun_theoretical} km/s')

Theoretical orbital speed of the Earth around Sun: 30.028076027046918 km/s


In [38]:
# A second check:
# The angular difference between the autumn equinox and today's position vector of the Earth
# (in this tutorial October) should be in degrees the number of days passed the 22th September.
# Again please note: we use the "today" function to determine the Earth's state vector.
# Now the "autumn vector" is simpley (1, 0, 0) in ECLIPJ2000 and we use this as a quick and simple
# rough estimation / computation

# Position vector
earth_position_wrt_sun = earth_state_wrt_sun[:3]

# Normalize it
earth_position_wrt_sun_norm = earth_position_wrt_sun / earth_sun_distance
print(earth_position_wrt_sun_norm)

# Define the "autumn vector" of the Earth
earth_position_wrt_sun_norm_autumn = np.array([1.0,0.0,0.0])

[ 2.15936478e-02  9.99766829e-01 -5.12448524e-05]


In [44]:
angular_distance_deg = np.degrees(np.arccos(
                        np.dot(earth_position_wrt_sun_norm, 
                        earth_position_wrt_sun_norm_autumn)))
print(f'Angular distance between autumn and todays position ({today_utc}) in degrees: {angular_distance_deg}')

Angular distance between autumn and todays position (2022-12-21T00:00:00) in degrees: 88.76267894578662
